In [1]:
import os
import tabula
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
import requests
import traceback
import pandas as pd
from io import BytesIO

from utils import standardizeFuels, standardizeFields, createJoinKey, isURLValid

In [17]:
import requests
import json

# Define the URL
url = "https://wabi-us-north-central-e-primary-api.analysis.windows.net/public/reports/querydata"

# Define the headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:134.0) Gecko/20100101 Firefox/134.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "ActivityId": "356dbf88-c094-4b11-b4e9-c7354d0ef98e",
    "RequestId": "fa2a2b70-5f72-5d0d-7209-04bf7e9cc635",
    "X-PowerBI-ResourceKey": "7e7b7100-e833-47dc-aed7-bc4c664cfe33",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://app.powerbi.com",
    "Connection": "keep-alive",
    "Referer": "https://app.powerbi.com/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site",
}

# Define the payload
data = {
    "version": "1.0.0",
    "queries": [
        {
            "Query": {
                "Commands": [
                    {
                        "SemanticQueryDataShapeCommand": {
                            "Query": {
                                "Version": 2,
                                "From": [
                                    {"Name": "g", "Entity": "Generator Interconnection Requests", "Type": 0},
                                    {"Name": "i", "Entity": "Interconnection Process", "Type": 0}
                                ],
                                # Add the remaining fields from your payload here...
                            },
                            "Binding": {
                                # Add binding fields here...
                            },
                            "DataReduction": {"DataVolume": 3, "Primary": {"Window": {"Count": 500}}},
                            "Version": 1,
                        },
                        "ExecutionMetricsKind": 1,
                    }
                ]
            },
            "CacheKey": "...",
            "QueryId": "",
            "ApplicationContext": {
                "DatasetId": "a038ae61-b404-4435-ba18-b8030c99e95c",
                "Sources": [
                    {"ReportId": "01fc9ed9-3f75-4a40-a52e-509a528f2cd0", "VisualId": "f881c9988772bbea355b"}
                ],
            },
        }
    ],
    "cancelQueries": [],
    "modelId": 2184566,
}

# Send the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print the response
print(response.status_code)
print(response.json())


200
{'jobIds': ['e8566cdd-1a6f-4dc3-8ff9-73273334850d'], 'results': [{'jobId': 'e8566cdd-1a6f-4dc3-8ff9-73273334850d', 'result': {'data': {'timestamp': '2025-01-11T15:45:21.301Z', 'rootActivityId': '9af9aa10-ffc3-47e1-849a-3d19f0673505', 'fromCache': False, 'dsr': {'DataShapes': [{'odata.error': {'code': 'rsInternalError', 'source': 'Unknown', 'message': {'lang': 'en-US', 'value': "Data Shape Query generation failed with error code: 'InternalDataShapeGenerationError'.Check the report server logs for more information."}, 'azure:values': [{'timestamp': '2025-01-11T15:45:21.3017893Z'}]}}]}}}}]}


In [27]:

url = "https://wabi-us-north-central-e-primary-api.analysis.windows.net/public/reports/querydata"

response = requests.post(url)

print("Status Code:", response.status_code)
print("Response:", response.text)

Status Code: 500
Response: { "statusCode": 500, "message": "Internal server error", "activityId": "88c6a7f0-2903-48d8-b5db-d48b29a1fc96" }


In [28]:
import requests
import json

# URL for the API endpoint
url = "https://wabi-us-north-central-e-primary-api.analysis.windows.net/public/reports/querydata"

# Load your JSON data (you mentioned you have it saved on your computer)
with open("C:/Users/zleig/Downloads/post.json", "r") as file:
    data = json.load(file)

# Headers copied from the browser
headers = {
    "Host": "wabi-us-north-central-e-primary-api.analysis.windows.net",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:134.0) Gecko/20100101 Firefox/134.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "ActivityId": "d00c71aa-64f5-4ebc-beb4-529f4823b631",
    "RequestId": "06a4e6d1-18c8-dc18-dff3-ffe0f607b56d",
    "X-PowerBI-ResourceKey": "7e7b7100-e833-47dc-aed7-bc4c664cfe33",
    "Content-Type": "application/json;charset=UTF-8",
    "Content-Length": "8040",  # Normally you don't need to set Content-Length manually
    "Origin": "https://app.powerbi.com",
    "Connection": "keep-alive",
    "Referer": "https://app.powerbi.com/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site"
}

# Send the POST request with headers and JSON data
response = requests.post(url, headers=headers, json=data)

# Check the response status and content
print("Status Code:", response.status_code)
print("Response JSON:", response.json())


Status Code: 200
Response JSON: {'jobIds': ['56589789-5591-4d53-bfc4-d54e5e1c0499'], 'results': [{'jobId': '56589789-5591-4d53-bfc4-d54e5e1c0499', 'result': {'data': {'timestamp': '2025-01-11T16:43:01.851Z', 'rootActivityId': '6edb867e-e8ae-4edf-8480-6c4d8d64f40b', 'descriptor': {'Select': [{'Kind': 1, 'Depth': 0, 'Value': 'G0', 'GroupKeys': [{'Source': {'Entity': 'Generator Interconnection Requests', 'Property': 'Request'}, 'Calc': 'G0', 'IsSameAsSelect': True}], 'Name': 'Generator Interconnection Requests.Request'}, {'Kind': 1, 'Depth': 0, 'Value': 'G1', 'Format': 'd', 'GroupKeys': [{'Source': {'Entity': 'Generator Interconnection Requests', 'Property': 'Queue Date'}, 'Calc': 'G1', 'IsSameAsSelect': True}], 'Name': 'Generator Interconnection Requests.Queue Date'}, {'Kind': 1, 'Depth': 0, 'Value': 'G2', 'GroupKeys': [{'Source': {'Entity': 'Generator Interconnection Requests', 'Property': 'Service'}, 'Calc': 'G2', 'IsSameAsSelect': True}], 'Name': 'Generator Interconnection Requests.Se

In [39]:


a = response.json()
b = a['results']

with open('C:/Users/zleig/Downloads/test.json', 'w') as json_file:
    json.dump(a, json_file, indent=4)  # 'indent=4' makes the file more readable


In [ ]:
os.chdir('../../gi-queue-map/')
print(os.getcwd())

In [40]:
from pandas import json_normalize

# Assuming 'data' is your nested dictionary
flattened_data = json_normalize(b, sep='_')
print(flattened_data)

                                  jobId     result_data_timestamp  \
0  56589789-5591-4d53-bfc4-d54e5e1c0499  2025-01-11T16:43:01.851Z   

             result_data_rootActivityId  \
0  6edb867e-e8ae-4edf-8480-6c4d8d64f40b   

                       result_data_descriptor_Select  \
0  [{'Kind': 1, 'Depth': 0, 'Value': 'G0', 'Group...   

  result_data_descriptor_Expressions_Primary_Groupings  \
0  [{'Keys': [{'Source': {'Entity': 'Generator In...     

   result_data_descriptor_Version result_data_metrics_Version  \
0                               2                       1.0.0   

                          result_data_metrics_Events  result_data_fromCache  \
0  [{'Id': 'c091a2cc-2c4d-4ad6-80db-d964043170e7'...                  False   

   result_data_dsr_Version  result_data_dsr_MinorVersion  \
0                        2                             1   

                                  result_data_dsr_DS  
0  [{'N': 'DS0', 'PH': [{'DM0': [{'S': [{'N': 'G0...  


In [3]:
def findNewURL(utility):
    #Access the download settings that track working urls and data updates
    ds_path = f'scripts/script_data/download_settings.csv'
    download_settings = pd.read_csv(ds_path, index_col='name')

    #Read data from download settings
    base_url = download_settings.loc[utility]['base_url']
    last_updated = '12/11/2024'
    #last_updated = download_settings.loc[utility]['last_updated']
    print(last_updated)
    date_format = download_settings.loc[utility]['date_format']

    #Convert tracker to datetime object
    date_tracker = datetime.strptime(last_updated, "%m/%d/%Y")

    #If the URL on file is already working, return None to indicate no update is necessary
    if isURLValid(base_url.format(date_tracker.strftime(date_format))):
        print("URL is still valid")
        return None
    print("Looking for new url")
    #If the URL on file does not work anymore, loop through all possible dates to find something new
    while date_tracker < datetime.now():
        #Create URL for testing
        formatted_date = date_tracker.strftime(date_format)
        full_url = base_url.format(formatted_date)

        #If the URL is valid, that means that the dataset needs to be updated
        if isURLValid(full_url):
            correct_date = date_tracker.strftime("%m/%d/%Y")
            #Update download settings to reflect changes
            download_settings.loc[utility, 'last_updated'] = correct_date
            #Save changes
            download_settings.to_csv(ds_path)
            #Return the new URL so the module can update the data
            return full_url
        else:
            #If no valid URL found, try the next day
            date_tracker = date_tracker + timedelta(days=1)
    print("Attention needed for " + utility, "No valid link found")
    return None



In [4]:
def importDuke(url):
    url = url
    response = requests.get(url)
    excel_file = BytesIO(response.content)
    # The active projects are in the first sheet
    # Gets read by default
    duke_df = pd.read_excel(excel_file, engine='openpyxl')
    header_row_index = duke_df[duke_df.iloc[:, 1] == "OPCO"].index[0]

    # Set that row as the header
    df_cleaned = duke_df.iloc[header_row_index + 1:-2].reset_index(drop=True)  # Rows below the header row
    df_cleaned.columns = duke_df.iloc[header_row_index]  # Set the column names
    return df_cleaned

In [5]:
dep_url = findNewURL('DEP')
dec_url = findNewURL('DEC')

dep_df = importDuke(dep_url)
dec_df = importDuke(dec_url)

duke_df = pd.concat([dec_df, dep_df])
#duke_df

12/11/2024
Looking for new url
12/11/2024
Looking for new url


In [9]:
duke_df = duke_df[~duke_df['Operational Status'].isin(["Withdrawn", "Commercial Operation - Commercial Operation Date Declared"])]

### Pretend like this is a standalone function

In [14]:
def importDuke(url):
    response = requests.get(url)
    excel_file = BytesIO(response.content)
    # The active projects are in the first sheet
    # Gets read by default
    duke_df = pd.read_excel(excel_file, engine='openpyxl')
    header_row_index = duke_df[duke_df.iloc[:, 1] == "OPCO"].index[0]

    # Set that row as the header
    df_cleaned = duke_df.iloc[header_row_index + 1:-2].reset_index(drop=True)  # Rows below the header row
    df_cleaned.columns = duke_df.iloc[header_row_index]  # Set the column names
    return df_cleaned

def getDukeQueue():
    dep_url = findNewURL('DEP')
    dec_url = findNewURL('DEC')

    if (dep_url is None) or (dec_url is None):
        duke_backup = pd.read_csv('data/individual_queues/duke_active_projects.csv')
        return duke_backup
    else:
        dep_df = importDuke(dep_url)
        dec_df = importDuke(dec_url)

        duke_df = pd.concat([dec_df, dep_df])

        ### Begin clean up ###
        duke_df = duke_df[~duke_df['Operational Status'].isin(["Withdrawn", "Commercial Operation - Commercial Operation Date Declared"])]
        duke_df['POI'] = duke_df['Transmission Line'].astype(str) + duke_df['Substation Name'].astype(str)
        return duke_df
getDukeQueue()


12/11/2024
Looking for new url
12/11/2024
Looking for new url


,NaN,OPCO,Transmission Designation,Source System Unique ID,Queue Number,Queue Issued Date,Interconnection Customer,Queue Indicator*,Operational Status,Operational Date,...,Project Size Winter MWAC,Energy Source Type,Facility County,Facility State,Transmission Line,Substation Name,Duke Estimated Startup Date,Type of Service,Advance Notice of Scoping Meeting,POI
0,NaN,DEC,FERC,962624,2024-09-27 14:15:00,2024-09-27 00:00:00,Masked,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,74.9,Solar,Greenwood,SC,Florida 44 kV,NaN,NaN,Both,NaN,Florida 44 kVnan
1,NaN,DEC,FERC,963122,2024-09-26 10:42:00,2024-09-26 00:00:00,Duke Energy Carolinas LLC,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,50,Solar,Laurens,SC,NaN,NaN,NaN,NRIS,NaN,nannan
2,NaN,DEC,FERC,962913,2024-09-26 12:05:00,2024-09-26 00:00:00,Masked,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,120000,Solar,McCormick,SC,NaN,NaN,NaN,NRIS,NaN,nannan
3,NaN,DEC,FERC,962989,2024-09-24 11:23:00,2024-09-24 00:00:00,Duke Energy Carolinas LLC,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,34,Solar,Anderson,SC,NaN,NaN,NaN,NRIS,NaN,nannan
4,NaN,DEC,FERC,962640,2024-09-24 11:30:00,2024-09-24 00:00:00,Duke Energy Carolinas LLC,2024 Solar + Procurement,Phase 1 Cluster Study - Pending,NaN,...,130,Solar,Stokes,NC,NaN,NaN,NaN,NRIS,NaN,nannan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,NaN,DEP,FERC,049220,2019-10-08 11:39:00,2019-10-08 00:00:00,"Duke Energy Progress, LLC",Transitional Serial Study,Engineering Design - In Progress,NaN,...,17.25,Battery,Buncombe,NC,NaN,NaN,2026-03-31 00:00:00,NaN,NaN,nannan
98,NaN,DEP,FERC,038056,2019-06-20 14:45:00,2019-06-20 00:00:00,"Knightdale Energy Center, LLC",Transitional Serial Study,Engineering Design - Pending,NaN,...,100,Battery,Wake,NC,Milburnie - Wake 230 kV,NaN,NaN,Both,NaN,Milburnie - Wake 230 kVnan
118,NaN,DEP,FERC,018124,Q442,2017-10-26 00:00:00,"Duke Energy Progress, LLC",Transitional Serial Study,Commissioning - Permission to Operate at Full ...,2023-03-01 00:00:00,...,11,Battery,Onslow,NC,NaN,NaN,NaN,NaN,NaN,nannan
126,NaN,DEP,FERC,948720,2017-06-16 00:00:00,2017-06-16 00:00:00,"Duke Energy Progress, LLC",NaN,Construction - In Progress,NaN,...,80000,Solar,Nash,NC,NaN,NaN,2025-07-31 00:00:00,NRIS,NaN,nannan
